In [2]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Desired input shape for InceptionV3
input_shape = (299, 299, 3)

In [3]:
# ImageDataGenerator for rescaling and resizing and data_augmentation
datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to the range [0, 1]
    validation_split=0.2,  # Optional: Split data into training and validation sets
    preprocessing_function=lambda x: tf.image.resize(x, (input_shape[0], input_shape[1])),
    rotation_range=20,  # Rotate images by a random angle within the specified range
    width_shift_range=0.2,  # Shift images horizontally by a fraction of total width
    height_shift_range=0.2,  # Shift images vertically by a fraction of total height
    shear_range=0.2,  # Apply shear transformation
    zoom_range=0.2,  # Zoom in/out on images
    horizontal_flip=True,  # Randomly flip images horizontally
    vertical_flip=True,
    brightness_range=[0.5, 1.5],  # Adjust brightness
#   contrast_range=[0.5, 1.5],  # Adjust contrast
    channel_shift_range=50,  # Shift channel values
    fill_mode='nearest'  # Fill in newly created pixels after rotation or shifting
)

In [4]:
# Path to your dataset
dataset_path = "C:/Users/user_99/Desktop/data_of_parikar"

In [5]:
# Data generators for training and validation
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # For training data
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # For validation data
)

Found 10239 images belonging to 12 classes.
Found 2555 images belonging to 12 classes.


In [6]:
# Pre-trained InceptionV3 model without the top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

In [7]:
# Freeze the convolutional base, unfreeze the last 3 convolutional layers
#Fine_tunning
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [8]:
# Create a new model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(12, activation='softmax')) # Final Dense layer with 12 nodes (number of classes) and softmax activation

In [9]:
# Compile the model
model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [10]:
# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 12)                3084      
                                                                 
Total params: 21,140,556
Trainable params: 11,145,484
Non-trainable params: 9,995,072
_________________________________________________________________


In [ ]:
# ModelCheckpoint callback to save the weights
checkpoint_filepath = "C:/Users/user_99/Desktop/weights_checkpoint.h5"
model_checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    save_weights_only=True,
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

In [ ]:
# Train the model using the data generators with the ModelCheckpoint callback
model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[model_checkpoint])

Epoch 1/10
 17/320 [>.............................] - ETA: 2:19 - loss: 2.3867 - accuracy: 0.2077

In [ ]:
# Specify the epoch from which you want to convert weights to TensorFlow Lite
#desired_epoch = 50  # Change this to the desired epoch

In [ ]:
# Convert the model to TensorFlow Lite with post-training quantization using the saved weights
# model.load_weights(checkpoint_filepath)

In [51]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_model_path = "C:/Users/user_99/Desktop/tflite_model.tflite"
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\user_99\AppData\Local\Temp\tmp5bgek5ud\assets


INFO:tensorflow:Assets written to: C:\Users\user_99\AppData\Local\Temp\tmp5bgek5ud\assets


In [3]:
# # Convert the model to TensorFlow Lite with post-training quantization
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# tflite_model_path = "C:/Users/user_99/Desktop/tflite_model.tflite"
# with open(tflite_model_path, 'wb') as f:
#     f.write(tflite_model)

NameError: name 'model' is not defined